# How to build Angular components?



In [1]:
// first install angular cli
var path = require('path');
var MemoryFileSystem = require("memory-fs");
var data = {
    'angular-cli.json': 
`
{
    "project": {
        "name": "act-ecommerce"
    },
    "apps": [
        {
            "root": "src",
            "main": "main.ts",
            "tsconfig": "../tsconfig.json",
            "prefix": "bc",
            "styles": [],
            "scripts": []
        }
    ]
}
`
};
var memfs = new MemoryFileSystem(data);
var importer = require('../Core');
var ng, interpret, interpretObject, mountAndRewireAngular;
var createComponent = (project, name) => {
    return importer.importNotebook('../Core/interpret all notebooks.ipynb')
        .then(r => {
            interpret = r['interpret'];
            interpretObject = r['interpretObject'];
        })
        .then(() => Promise.all([
            interpret('memory-fs rewire')
        ]))
        .then(r => interpretObject(r.map(r => r[0])))
        .then(r => {
            mountAndRewireAngular = eval("'use strict';" + r[0].code);
            var conf = {
                cliArgs: ['generate',
                          'component',
                          '--skip-import',
                          name],
                inputStream: process.stdin,
                outputStream: process.stdout
            };
            console.log(process.cwd());
            memfs.mkdirpSync(path.resolve(path.join(process.cwd(), '..')));
            memfs.writeFileSync(
                path.resolve(path.join(process.cwd(), '..', 'angular-cli.json')),
                data['angular-cli.json']);
            ng = mountAndRewireAngular(memfs);
            return conf;
        })
        // ng is mocked at this point
        .then(conf => ng(conf))
        .then(r => {
            console.log(r);
            return data['C:']['Users']['brian.cullinan']['Documents']['jupytangular2']['src']['app']['testing'];
        });
        // unmock ng
};

// obviously we can generate and use a CLI interface from within Node.js

try {
$$.async();
createComponent('portal', 'testing')
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));
} catch (e) {
    $$.sendError(e);
}





C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks
installing component
  create src\app\testing\testing.component.css
  create src\app\testing\testing.component.html
  create src\app\testing\testing.component.spec.ts
  create src\app\testing\testing.component.ts
1


Path must be a string. Received undefined
TypeError: Path must be a string. Received undefined
    at assertPath (path.js:7:11)
    at Object.dirname (path.js:697:5)
    at Class.afterInstall (C:\Users\brian.cullinan\Documents\jupytangular2\node_modules\@angular\cli\blueprints\component\index.js:214:49)
    at tryCatch (C:\Users\brian.cullinan\Documents\jupytangular2\node_modules\rsvp\dist\rsvp.js:525:12)
    at invokeCallback (C:\Users\brian.cullinan\Documents\jupytangular2\node_modules\rsvp\dist\rsvp.js:538:13)
    at publish (C:\Users\brian.cullinan\Documents\jupytangular2\node_modules\rsvp\dist\rsvp.js:508:7)
    at flush (C:\Users\brian.cullinan\Documents\jupytangular2\node_modules\rsvp\dist\rsvp.js:2400:5)
    at _combinedTickCallback (internal/process/next_tick.js:67:7)
    at process._tickCallback (internal/process/next_tick.js:98:9)


{ '': true,
  'angular-cli.json': <Buffer 0a 7b 0a 20 20 20 20 22 70 72 6f 6a 65 63 74 22 3a 20 7b 0a 20 20 20 20 20 20 20 20 22 6e 61 6d 65 22 3a 20 22 61 63 74 2d 65 63 6f 6d 6d 65 72 63 65 ... >,
  src: { '': true, app: { '': true, testing: [Object] } } }

In [2]:
var execSync = require('child_process').execSync();
try {
    require.resolve('glob');
} catch (e) {
    execSync('glob');
}
var glob = require('glob');
var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;

// But we also want to automatically load projects?
var listProjects = (root = PROFILE_PATH) => {
    return new Promise((resolve, reject) => {
        glob('**/package.json', {
            ignore: ['**/node_modules/**',
                    '**/vendor/**'],
            cwd: root,
            silent: true,
            nodir: true,
            strict: false
        }, (err, matches) => {
            var result = {};
            matches.forEach(m => {
                var projectPath = path.dirname(path.join(root, m));
                result[path.basename(projectPath)] = projectPath;
            })
            return resolve(result);
        });
    });
};
(listProjects);


In [4]:
$$.async();
listProjects(path.join(PROFILE_PATH, 'Documents'))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));

In [ ]:
// Wouldn't it be nice if we could list these projects as interactive icons?

